#FA1 - Heights Dataset - Data Wrangling

#Importing Libraries

In [1]:
import pandas as pd

#Loading Data

In [15]:
df = pd.read_csv('heights.csv')
df.head()

,earn,height,sex,ed,age,race
0,50000.0,74.424439,male,16,45,white
1,60000.0,65.537543,female,16,58,white
2,30000.0,63.629198,female,16,29,white
3,50000.0,63.108562,female,16,91,other
4,51000.0,63.402484,female,17,39,white


#Preprocessing

##Check for missing values


In [11]:
missing_values = df.isnull().sum()
missing_values

earn      0
height    0
sex       0
ed        0
age       0
race      0
dtype: int64

##Check for dupes

In [12]:
duplicate_rows = df.duplicated().sum()
duplicate_rows

0

In [13]:
df.describe()

,earn,height,ed,age
count,1192.000000,1192.000000,1192.000000,1192.000000
mean,23154.773490,66.915154,13.504195,41.378356
std,19472.296925,3.853968,2.420175,15.867428
min,200.000000,57.503219,3.000000,18.000000
25%,10000.000000,64.009746,12.000000,29.000000
50%,20000.000000,66.451265,13.000000,38.000000
75%,30000.000000,69.848100,16.000000,51.000000
max,200000.000000,77.051282,18.000000,91.000000


#Pivoting

##Wide to Long

In [14]:
long_format = pd.melt(df, id_vars=['sex', 'age', 'race'], value_vars=['earn', 'height', 'ed'], var_name='variable', value_name='value')
long_format.head()


,sex,age,race,variable,value
0,male,45,white,earn,50000.0
1,female,58,white,earn,60000.0
2,female,29,white,earn,30000.0
3,female,91,other,earn,50000.0
4,female,39,white,earn,51000.0


##Long to Wide

In [16]:
wide_format = long_format.pivot_table(index=['sex', 'age', 'race'], columns='variable', values='value', aggfunc='first').reset_index()
wide_format.head()

variable,sex,age,race,earn,ed,height
0,female,18,black,600.0,12.0,70.238590
1,female,18,white,15000.0,12.0,68.456542
2,female,19,black,1000.0,16.0,63.087134
3,female,19,hispanic,13000.0,12.0,60.817039
4,female,19,white,8000.0,13.0,68.291097


#Splitting

In [18]:
demographic_info = df[['sex', 'age', 'race']]
other_info = df[['earn', 'height', 'ed']]
demographic_info.head(), other_info.head()

(      sex  age   race
 0    male   45  white
 1  female   58  white
 2  female   29  white
 3  female   91  other
 4  female   39  white,
       earn     height  ed
 0  50000.0  74.424439  16
 1  60000.0  65.537543  16
 2  30000.0  63.629198  16
 3  50000.0  63.108562  16
 4  51000.0  63.402484  17)

#Joining

To join them back, we need a common identifier; we'll use the index of the DataFrame as a makeshift key

In [20]:
demographic_info['key'] = demographic_info.index
other_info['key'] = other_info.index


<ipython-input-20-fdea184ed7d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demographic_info['key'] = demographic_info.index
<ipython-input-20-fdea184ed7d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_info['key'] = other_info.index


Joining back then dropping key column

In [21]:
joined_data = pd.merge(demographic_info, other_info, on='key')
joined_data.drop('key', axis=1, inplace=True)
joined_data.head()

,sex,age,race,earn,height,ed
0,male,45,white,50000.0,74.424439,16
1,female,58,white,60000.0,65.537543,16
2,female,29,white,30000.0,63.629198,16
3,female,91,other,50000.0,63.108562,16
4,female,39,white,51000.0,63.402484,17
